In [1]:
import warnings
warnings.resetwarnings()

import scprep
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
import random

import os

from tqdm import tqdm

from deepimpute.multinet import MultiNet

from sklearn.metrics import mean_absolute_percentage_error as mape
from sklearn.metrics import mean_squared_error as mse

2023-09-22 14:28:03.597594: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-22 14:28:08.312551: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
def get_data_for_i(i):
    original_ = pd.read_csv('../../data/1k_cell//data.csv', index_col=0)
    df_ = pd.read_csv('../../data/1k_cell/drp_{}0.csv'.format(i), index_col=0)
    df_.index = [int(i) for i in df_.index]
    df_.columns = [int(i) for i in df_.columns]

    original_.columns = df_.columns
    original_.index = df_.index

    n = original_.size
    original_val = original_.values.copy()
    t = list(np.ndindex(original_.shape))
    random.Random(42).shuffle(t)

    mask = t[:int(len(t)/10 * i)]

    thr = np.sum(np.sign(df_)) > 0
    original_ = original_.loc[:, list(thr)]
    df_ = df_.loc[:, list(thr)]

    # original = original_.values
    original = np.log(original_+1)

    # df = df_.values
    df = np.log(df_+1)

    tmp = pd.DataFrame(thr)
    remove = [int(i) for i in tmp[tmp[0] == False].index]
    mask = [i for i in mask if i[1] not in remove]
    
    return df_, mask, original

In [3]:
def get_cos_sim(vector1, vector2):
    dot_product = np.dot(vector1, vector2)

    # ベクトルの大きさ（ノルム）を計算します
    norm_vector1 = np.linalg.norm(vector1)
    norm_vector2 = np.linalg.norm(vector2)

    # Cosine Similarityを計算します
    cosine_similarity = dot_product / (norm_vector1 * norm_vector2)
    
    return cosine_similarity

In [4]:
def get_l1norm(vector1, vector2):
    l1_distance = np.sum(np.abs(vector1 - vector2))
    return l1_distance

In [6]:
mses = {}
corrs = {}
coss = {}
l1 = {}
method = 'DeepImpute'

for i in (range(9)):
    print(i)
    df_, mask, original = get_data_for_i(i+1)

    multinet = MultiNet() 
    multinet.fit(df_)
    
    pred = multinet.predict(df_)
    pred = np.log(pred.values+1)
    
    pred = pd.DataFrame(pred, index=df_.index, columns=df_.columns)
    origin = np.array([original.loc[i] for i in mask])
    predict = np.array([pred.loc[i] for i in mask])

    mses[i] = mse(origin, predict)
    corrs[i] = np.corrcoef(origin, predict)[0][1]
    coss[i] = get_cos_sim(origin, predict) 
    l1[i] = get_l1norm(origin, predict)

0
Using all the cores (20)
Input dataset is 1000 cells (rows) and 12345 genes (columns)
First 3 rows and columns:
   0  1  2
0  2  3  0
1  1  1  0
2  0  3  0
7680 genes selected for imputation
Net 0: 1537 predictors, 512 targets
Net 1: 1578 predictors, 512 targets
Net 2: 1564 predictors, 512 targets
Net 3: 1553 predictors, 512 targets
Net 4: 1462 predictors, 512 targets
Net 5: 1551 predictors, 512 targets
Net 6: 1557 predictors, 512 targets
Net 7: 1553 predictors, 512 targets
Net 8: 1512 predictors, 512 targets
Net 9: 1526 predictors, 512 targets
Net 10: 1589 predictors, 512 targets
Net 11: 1585 predictors, 512 targets
Net 12: 1541 predictors, 512 targets
Net 13: 1587 predictors, 512 targets
Net 14: 1602 predictors, 512 targets
Normalization
Building network
[{'type': 'dense', 'neurons': 256, 'activation': 'relu'}, {'type': 'dropout', 'rate': 0.2}]


/home/inoue019/.local/lib/python3.10/site-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Fitting with 1000 cells
Epoch 1/500
15/15 [==============================] - 3s 55ms/step - loss: 6.7051 - dense_45_loss: 0.3635 - dense_46_loss: 0.4752 - dense_47_loss: 0.4475 - dense_48_loss: 0.4055 - dense_49_loss: 0.4754 - dense_50_loss: 0.4000 - dense_51_loss: 0.4159 - dense_52_loss: 0.5390 - dense_53_loss: 0.4668 - dense_54_loss: 0.3860 - dense_55_loss: 0.4797 - dense_56_loss: 0.4211 - dense_57_loss: 0.5274 - dense_58_loss: 0.4158 - dense_59_loss: 0.4862 - val_loss: 4.4386 - val_dense_45_loss: 0.2582 - val_dense_46_loss: 0.2961 - val_dense_47_loss: 0.2730 - val_dense_48_loss: 0.2831 - val_dense_49_loss: 0.3053 - val_dense_50_loss: 0.2612 - val_dense_51_loss: 0.2812 - val_dense_52_loss: 0.3511 - val_dense_53_loss: 0.2946 - val_dense_54_loss: 0.2807 - val_dense_55_loss: 0.3220 - val_dense_56_loss: 0.2673 - val_dense_57_loss: 0.3378 - val_dense_58_loss: 0.2871 - val_dense_59_loss: 0.3398
Epoch 2/500
15/15 [==============================] - 0s 18ms/step - loss: 4.4912 - dense_45_loss

15/15 [==============================] - 0s 18ms/step - loss: 2.0003 - dense_45_loss: 0.1191 - dense_46_loss: 0.1360 - dense_47_loss: 0.1319 - dense_48_loss: 0.1302 - dense_49_loss: 0.1549 - dense_50_loss: 0.1274 - dense_51_loss: 0.1255 - dense_52_loss: 0.1407 - dense_53_loss: 0.1349 - dense_54_loss: 0.1281 - dense_55_loss: 0.1355 - dense_56_loss: 0.1288 - dense_57_loss: 0.1376 - dense_58_loss: 0.1236 - dense_59_loss: 0.1460 - val_loss: 1.5861 - val_dense_45_loss: 0.1005 - val_dense_46_loss: 0.1064 - val_dense_47_loss: 0.1028 - val_dense_48_loss: 0.1082 - val_dense_49_loss: 0.1207 - val_dense_50_loss: 0.1022 - val_dense_51_loss: 0.1016 - val_dense_52_loss: 0.1069 - val_dense_53_loss: 0.1068 - val_dense_54_loss: 0.1086 - val_dense_55_loss: 0.1017 - val_dense_56_loss: 0.1011 - val_dense_57_loss: 0.1003 - val_dense_58_loss: 0.1006 - val_dense_59_loss: 0.1178
Epoch 11/500
15/15 [==============================] - 0s 18ms/step - loss: 1.9522 - dense_45_loss: 0.1158 - dense_46_loss: 0.1327 - 

15/15 [==============================] - 0s 17ms/step - loss: 1.6847 - dense_45_loss: 0.1004 - dense_46_loss: 0.1149 - dense_47_loss: 0.1118 - dense_48_loss: 0.1100 - dense_49_loss: 0.1313 - dense_50_loss: 0.1089 - dense_51_loss: 0.1072 - dense_52_loss: 0.1175 - dense_53_loss: 0.1121 - dense_54_loss: 0.1070 - dense_55_loss: 0.1115 - dense_56_loss: 0.1092 - dense_57_loss: 0.1160 - dense_58_loss: 0.1053 - dense_59_loss: 0.1216 - val_loss: 1.4763 - val_dense_45_loss: 0.0933 - val_dense_46_loss: 0.0987 - val_dense_47_loss: 0.0957 - val_dense_48_loss: 0.0980 - val_dense_49_loss: 0.1136 - val_dense_50_loss: 0.0952 - val_dense_51_loss: 0.0946 - val_dense_52_loss: 0.1002 - val_dense_53_loss: 0.0997 - val_dense_54_loss: 0.1011 - val_dense_55_loss: 0.0939 - val_dense_56_loss: 0.0946 - val_dense_57_loss: 0.0948 - val_dense_58_loss: 0.0936 - val_dense_59_loss: 0.1094
Epoch 20/500
15/15 [==============================] - 0s 17ms/step - loss: 1.6576 - dense_45_loss: 0.0982 - dense_46_loss: 0.1131 - 

...

15/15 [==============================] - 0s 15ms/step - loss: 0.1549 - dense_196_loss: 0.0281 - dense_197_loss: 0.0253 - dense_198_loss: 0.0236 - dense_199_loss: 0.0244 - dense_200_loss: 0.0258 - dense_201_loss: 0.0276 - val_loss: 0.2841 - val_dense_196_loss: 0.0534 - val_dense_197_loss: 0.0446 - val_dense_198_loss: 0.0424 - val_dense_199_loss: 0.0433 - val_dense_200_loss: 0.0512 - val_dense_201_loss: 0.0492
Epoch 59/500
15/15 [==============================] - 0s 20ms/step - loss: 0.1534 - dense_196_loss: 0.0277 - dense_197_loss: 0.0252 - dense_198_loss: 0.0234 - dense_199_loss: 0.0242 - dense_200_loss: 0.0256 - dense_201_loss: 0.0272 - val_loss: 0.2837 - val_dense_196_loss: 0.0533 - val_dense_197_loss: 0.0443 - val_dense_198_loss: 0.0423 - val_dense_199_loss: 0.0432 - val_dense_200_loss: 0.0513 - val_dense_201_loss: 0.0492
Epoch 60/500
15/15 [==============================] - 0s 16ms/step - loss: 0.1510 - dense_196_loss: 0.0273 - dense_197_loss: 0.0249 - dense_198_loss: 0.0230 - dens

/home/inoue019/.local/lib/python3.10/site-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Fitting with 1000 cells
Epoch 1/500
15/15 [==============================] - 2s 28ms/step - loss: 0.7733 - dense_206_loss: 0.2099 - dense_207_loss: 0.2044 - dense_208_loss: 0.1716 - dense_209_loss: 0.1874 - val_loss: 0.6351 - val_dense_206_loss: 0.1637 - val_dense_207_loss: 0.1509 - val_dense_208_loss: 0.1648 - val_dense_209_loss: 0.1557
Epoch 2/500
15/15 [==============================] - 0s 9ms/step - loss: 0.7596 - dense_206_loss: 0.2062 - dense_207_loss: 0.2006 - dense_208_loss: 0.1685 - dense_209_loss: 0.1843 - val_loss: 0.6265 - val_dense_206_loss: 0.1614 - val_dense_207_loss: 0.1487 - val_dense_208_loss: 0.1628 - val_dense_209_loss: 0.1537
Epoch 3/500
15/15 [==============================] - 0s 9ms/step - loss: 0.7465 - dense_206_loss: 0.2028 - dense_207_loss: 0.1970 - dense_208_loss: 0.1655 - dense_209_loss: 0.1812 - val_loss: 0.6172 - val_dense_206_loss: 0.1589 - val_dense_207_loss: 0.1462 - val_dense_208_loss: 0.1606 - val_dense_209_loss: 0.1516
Epoch 4/500
15/15 [===========

Epoch 27/500
15/15 [==============================] - 0s 9ms/step - loss: 0.1366 - dense_206_loss: 0.0374 - dense_207_loss: 0.0332 - dense_208_loss: 0.0322 - dense_209_loss: 0.0338 - val_loss: 0.1549 - val_dense_206_loss: 0.0389 - val_dense_207_loss: 0.0355 - val_dense_208_loss: 0.0458 - val_dense_209_loss: 0.0348
Epoch 28/500
15/15 [==============================] - 0s 9ms/step - loss: 0.1333 - dense_206_loss: 0.0368 - dense_207_loss: 0.0320 - dense_208_loss: 0.0314 - dense_209_loss: 0.0331 - val_loss: 0.1539 - val_dense_206_loss: 0.0387 - val_dense_207_loss: 0.0353 - val_dense_208_loss: 0.0453 - val_dense_209_loss: 0.0345
Epoch 29/500
15/15 [==============================] - 0s 9ms/step - loss: 0.1299 - dense_206_loss: 0.0359 - dense_207_loss: 0.0315 - dense_208_loss: 0.0304 - dense_209_loss: 0.0322 - val_loss: 0.1531 - val_dense_206_loss: 0.0386 - val_dense_207_loss: 0.0353 - val_dense_208_loss: 0.0450 - val_dense_209_loss: 0.0343
Epoch 30/500
15/15 [==============================] 

Epoch 53/500
15/15 [==============================] - 0s 9ms/step - loss: 0.0832 - dense_206_loss: 0.0227 - dense_207_loss: 0.0206 - dense_208_loss: 0.0193 - dense_209_loss: 0.0206 - val_loss: 0.1473 - val_dense_206_loss: 0.0379 - val_dense_207_loss: 0.0348 - val_dense_208_loss: 0.0419 - val_dense_209_loss: 0.0328
Epoch 54/500
15/15 [==============================] - 0s 11ms/step - loss: 0.0819 - dense_206_loss: 0.0221 - dense_207_loss: 0.0204 - dense_208_loss: 0.0189 - dense_209_loss: 0.0204 - val_loss: 0.1473 - val_dense_206_loss: 0.0379 - val_dense_207_loss: 0.0348 - val_dense_208_loss: 0.0419 - val_dense_209_loss: 0.0327
Epoch 55/500
15/15 [==============================] - 0s 10ms/step - loss: 0.0806 - dense_206_loss: 0.0219 - dense_207_loss: 0.0200 - dense_208_loss: 0.0184 - dense_209_loss: 0.0203 - val_loss: 0.1472 - val_dense_206_loss: 0.0379 - val_dense_207_loss: 0.0347 - val_dense_208_loss: 0.0419 - val_dense_209_loss: 0.0327
Epoch 56/500
15/15 [==============================

In [7]:
pd.DataFrame(mses.values()).T

,0,1,2,3,4,5,6,7,8
0,0.604957,0.57672,0.551184,0.525814,0.469207,0.442816,0.394013,0.377357,0.341969


In [8]:
pd.DataFrame(corrs.values()).T

,0,1,2,3,4,5,6,7,8
0,0.433414,0.435247,0.437792,0.439685,0.452587,0.458513,0.475045,0.480136,0.488525


In [9]:
pd.DataFrame(coss.values()).T

,0,1,2,3,4,5,6,7,8
0,0.570743,0.572016,0.573306,0.573424,0.577999,0.579308,0.582698,0.581048,0.57042


In [10]:
pd.DataFrame(l1.values()).T

,0,1,2,3,4,5,6,7,8
0,702163.629571,1.331898e+06,1.893848e+06,2.388749e+06,2.637461e+06,2.964491e+06,3.016201e+06,3.226211e+06,3.151855e+06
